## 1. Import libraries and requirements

In [1]:
# Loading required packages
import spacy
import pandas as pd
import re
import gensim
import gensim.downloader
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
import nltk
from numpy import mean
from numpy import std
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import json
import csv
import ast
import numpy as np


# Select Spacy model
# Efficiency
# nlp = spacy.load("en_core_web_sm")

# Accuracy
# nlp = spacy.load("en_core_web_trf")

# Change working directory
%cd '/Users/richardfrissen/Documents/Maastricht University/Thesis/Development/Datasets'

/Users/richardfrissen/Documents/Maastricht University/Thesis/Development/Datasets


In [ ]:
columns = ['Token', 'Label', 'pos', 'ent_type', 'is_alpha', 'is_ascii', 'is_digit', 'is_lower', 'is_upper', 'is_title', 'is_punct', 'is_space', 'like_num', 'is_oov', 'is_stop', 'like_num', 'lang', 'sentiment']
data = pd.read_csv('EMSCAD/Output data/FULL_Annotation_data_output.tsv', sep='\t', header = None)

In [ ]:
#text length
#non-dictionary word counts
#unique word ratios
#part of speech(POS)
#

data = data.head(10000)

# data[0] = data[0].astype(str)

In [ ]:
%%time
def feature_extraction(input_column):
    features = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    for i in input_column:
        i = str(i)
        doc = nlp(i)
        word = i
        for token in doc:
            features[0].append(token.pos)
            features[1].append(token.ent_type)
            features[2].append(token.is_alpha)
            features[3].append(token.is_ascii)
            features[4].append(token.is_digit)
            features[5].append(token.is_lower)
            features[6].append(token.is_upper)
            features[7].append(token.is_title)
            features[8].append(token.is_punct)
            features[9].append(token.is_space)
            features[10].append(token.like_num)
            features[11].append(token.is_oov)
            features[12].append(token.is_stop)
            features[13].append(token.lang)
            features[14].append(token.sentiment)
            features[15].append(len(word))
    return features

features = feature_extraction(data[0])

In [ ]:
%%time
features = pd.DataFrame(features)
features = features.transpose()
data = pd.concat([data, features], axis=1)
data.columns = ['Token', 'Label', 'pos', 'ent_type', 'is_alpha', 'is_ascii', 'is_digit', 'is_lower', 'is_upper', 'is_title', 'is_punct', 'is_space', 'like_num', 'is_oov', 'is_stop', 'lang', 'sentiment', 'word_length']

data.head(10)

In [ ]:
# data.to_csv('EMSCAD/Text classification models/SVM/data_features_full_dataset.csv', index = False)

data = pd.read_csv('EMSCAD/Text classification models/SVM/data_features_full_dataset.csv', header = 0)
# data = data.head(10)
data = data.drop(columns=['Unnamed: 0'])
data

In [ ]:
word2vec = gensim.downloader.load('word2vec-google-news-300')

In [ ]:
%%time

def word2vec_wordembed(input_column):
    word2vec_result =[]
    for i in input_column:
        try:
            i = str(i)
            result_array = word2vec[i]
            result_list = result_array.tolist()
            word2vec_result.append(result_list)
        except KeyError:
            word2vec_result.append(np.nan)
        except TypeError: 
            word2vec_result.append(np.nan)


    return word2vec_result
        
word_embedding = word2vec_wordembed(data['Token'])

In [ ]:
len(word_embedding)

In [ ]:
%%time

word_embedding_df = pd.DataFrame(word_embedding)
word_embedding_series = word_embedding_df[0].apply(pd.Series)
data = pd.concat([data, word_embedding_series], axis=1)

In [ ]:
data.head(10)

In [ ]:
%%time

word_embedding = pd.DataFrame(word_embedding)
word_embedding = word_embedding[0].apply(pd.Series)
data = pd.concat([data, word_embedding], axis=1)

In [2]:
%%time

# data.to_csv('EMSCAD/Text classification models/SVM/data_features_full_wordembedding.csv', index = False)


data = pd.read_csv('EMSCAD/Text classification models/SVM/data_features_full_wordembedding.csv', nrows=1000000, na_values=['nan'])

data.head(10)

CPU times: user 44.7 s, sys: 4.11 s, total: 48.9 s
Wall time: 49.7 s


,Token,Label,pos,ent_type,is_alpha,is_ascii,is_digit,is_lower,is_upper,is_title,...,290,291,292,293,294,295,296,297,298,299
0,Food52,O,92,0,False,True,False,False,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a,O,90,0,True,True,False,True,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,fast,O,86,0,True,True,False,True,False,False,...,0.177734,-0.012024,-0.123047,0.086914,0.071289,0.049561,-0.032959,0.192383,0.037109,-0.110352
3,growing,O,100,0,True,True,False,True,False,False,...,-0.007233,0.189453,-0.218750,0.060547,-0.039551,-0.050537,-0.058838,0.170898,0.253906,-0.277344
4,James,O,96,0,True,True,False,False,False,True,...,-0.029297,-0.029297,-0.028931,0.208008,0.028320,-0.244141,-0.015564,-0.070312,0.011963,0.003143
5,Beard,O,96,0,True,True,False,False,False,True,...,0.071777,-0.159180,-0.060791,0.347656,0.250000,-0.285156,0.125000,-0.067383,0.202148,0.226562
6,Award,O,92,0,True,True,False,False,False,True,...,-0.062988,0.016968,-0.734375,-0.169922,-0.102051,-0.106934,-0.030029,0.462891,0.144531,0.421875
7,winning,O,100,0,True,True,False,True,False,False,...,0.034668,-0.020996,-0.187500,-0.008057,0.210938,-0.001945,0.210938,-0.263672,-0.062012,-0.118164
8,online,O,86,0,True,True,False,True,False,False,...,0.063965,-0.064453,-0.112305,0.108398,0.103027,-0.042480,0.166992,0.131836,-0.002625,-0.130859
9,food,O,92,0,True,True,False,True,False,False,...,-0.369141,-0.055176,-0.149414,0.398438,0.092773,0.033936,-0.061523,0.159180,0.127930,0.002228


In [3]:
%%time

data = data.replace(np.nan, '0', regex=True)
data.drop('Token', axis=1, inplace=True)
data.head(10)

CPU times: user 40.1 s, sys: 1min 29s, total: 2min 9s
Wall time: 2min 31s


,Label,pos,ent_type,is_alpha,is_ascii,is_digit,is_lower,is_upper,is_title,is_punct,...,290,291,292,293,294,295,296,297,298,299
0,O,92,0,False,True,False,False,False,True,False,...,0,0,0,0,0,0,0,0,0,0
1,O,90,0,True,True,False,True,False,False,False,...,0,0,0,0,0,0,0,0,0,0
2,O,86,0,True,True,False,True,False,False,False,...,0.177734,-0.012024,-0.123047,0.086914,0.071289,0.049561,-0.032959,0.192383,0.037109,-0.110352
3,O,100,0,True,True,False,True,False,False,False,...,-0.007233,0.189453,-0.21875,0.060547,-0.039551,-0.050537,-0.058838,0.170898,0.253906,-0.277344
4,O,96,0,True,True,False,False,False,True,False,...,-0.029297,-0.029297,-0.028931,0.208008,0.02832,-0.244141,-0.015564,-0.070312,0.011963,0.003143
5,O,96,0,True,True,False,False,False,True,False,...,0.071777,-0.15918,-0.060791,0.347656,0.25,-0.285156,0.125,-0.067383,0.202148,0.226562
6,O,92,0,True,True,False,False,False,True,False,...,-0.062988,0.016968,-0.734375,-0.169922,-0.102051,-0.106934,-0.030029,0.462891,0.144531,0.421875
7,O,100,0,True,True,False,True,False,False,False,...,0.034668,-0.020996,-0.1875,-0.008057,0.210938,-0.001945,0.210938,-0.263672,-0.062012,-0.118164
8,O,86,0,True,True,False,True,False,False,False,...,0.063965,-0.064453,-0.112305,0.108398,0.103027,-0.04248,0.166992,0.131836,-0.002625,-0.130859
9,O,92,0,True,True,False,True,False,False,False,...,-0.369141,-0.055176,-0.149414,0.398438,0.092773,0.033936,-0.061523,0.15918,0.12793,0.002228


In [4]:
%%time

# 80% / 20% split
# Train, Test = train_test_split(data1, test_size=0.2, shuffle=False)

X = data.drop(['Label'],axis=1).values # independant features
y = data['Label'].values # dependant variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

CPU times: user 21 s, sys: 1min 6s, total: 1min 27s
Wall time: 1min 49s


In [5]:
len(y_test)

200000

In [6]:
del(data)

# Encoder = preprocessing.LabelEncoder()
# y_train = Encoder.fit_transform(y_train)
# y_test = Encoder.fit_transform(y_test)

In [7]:
X_train.shape, y_train.shape

((800000, 316), (800000,))

In [8]:
X_test.shape, y_test.shape

((200000, 316), (200000,))

In [11]:
%%time

clf = svm.SVC(kernel='linear')
scores1 = cross_val_score(clf, X_train, y_train, cv=10)
print(scores1)

[0.9949    0.9949    0.9949    0.9948875 0.9948875 0.9948875 0.9948875
 0.9948875 0.9948875 0.9948875]
CPU times: user 3h 18min 58s, sys: 2min 26s, total: 3h 21min 24s
Wall time: 3h 22min 8s


In [ ]:
# %%time

# clf = svm.LinearSVC(multi_class="crammer_singer")
# scores2 = cross_val_score(clf, X_train, y_train, cv=10)
# print(scores2)

In [13]:
# %%time

# clf = LogisticRegression()
# scores3 = cross_val_score(clf, X_train, y_train, cv=10)
# print(scores3)

In [10]:
%%time

clf = DecisionTreeClassifier()
scores4 = cross_val_score(clf, X_train, y_train, cv=10)
print(scores4)

[0.999925  0.9999625 0.9999375 0.9998875 0.99985   0.99995   0.9999375
 0.999925  0.9999625 0.9999   ]
CPU times: user 47min 48s, sys: 1min 40s, total: 49min 28s
Wall time: 49min 58s


In [12]:
import datetime
now = datetime.datetime.now()
print ("Current date and time : ")
print (now.strftime("%Y-%m-%d %H:%M:%S"))

Current date and time : 
2021-05-28 16:28:05


In [ ]:
3 4 1

In [ ]:
import numpy as np
from sklearn.model_selection import KFold

kf = KFold(n_splits=10, shuffle = False)
for train, test in kf.split(data):
    print("%s %s" % (train, test))
    
len(test)

In [ ]:
# initialize the model
model = svm.LinearSVC(multi_class="crammer_singer")

In [ ]:
%%time

i = 1
for train_index, test_index in kf.split(data):
    X_train = data.iloc[train].loc[:, features]
    X_test = data.iloc[test][features]
    y_train = data.iloc[train].loc[:,'Label']
    y_test = data.loc[test]['Label']
        
    #Train the model
    model.fit(X_train, y_train) #Training the model
    print(f"Accuracy for the fold no. {i} on the test set: {accuracy_score(y_test, model.predict(X_test))}")
    i += 1

In [ ]:
word2vec.?('supercalifragilisticexpialidocious')

In [ ]:
# wv = model.load("word2vec-google-news-300", mmap='r')

In [ ]:
gensim?

In [ ]:
data['Token'] = data['Token'].astype(str)

In [ ]:
sentences = data['Token']
model = Word2Vec(sentences)

In [ ]:
# evaluate a logistic regression model using k-fold cross-validation
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
# create dataset
X, y = make_classification(n_samples=100, n_features=20, n_informative=15, n_redundant=5, random_state=1)
# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# create model
model = LogisticRegression()
# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))